In [ ]:
#-----------------------------------------------------------------------------------------------------
# BERT와 sLLM 모델을 이용한 질의응답 서비스 구축 예
# - sLLM_test_embed.ipynb에서 임베딩된 문서들을 이용하여 프롬프트 생성 및 입력하는 예시임.
#
# 질의 응답 시스템 과정
# 문서들 전처리 : 
#    단락별루 분할(\n\n) - 불용어 제거 -문장별루 분할.
# 임베딩 : 
#    kpf-sbert-v1.1로  문장 평균 임베딩벡터 구함 - es에 문장별루 단락text와 평균벡터 저장.
# 프롬프트생성 및 입력 : 
#   검색어 입력(회사:과장일때 휴가 일수는 얼마?)-bert로 임베딩 검색(*스코어가 0.6이상인 경우 체택)-sLLM에 검색된 단락 text를 문맥으로 해서 prompt 구성
#   sLLM에 prompt 입력-응답 결과 출력
#-----------------------------------------------------------------------------------------------------
import os
import random
import numpy as np
import pandas as pd
import time
import random

import sys
sys.path.append('..')
from myutils import seed_everything, GPU_info, mlogging

LOGGER = mlogging(loggername="sllm-test", logfilename='../../log/sll-test.txt') # 로그

# param-----------------------------------
SEED = 111
seed_everything(SEED)
DEVICE = GPU_info() # GPU 혹은 CPU
OUT_DIMENSION = 128   # 출력 dimension 128 혹은 0(768)
EMBEDDING_METHOD=1 # 1=평균
NUM_CLUSTERS=10
NUM_CLUSTERS_VARIABLE=False
   
BATCH_SIZE=20
FLOAT_TYPE = 'float16'

# ES 접속
ES_URL = 'http://10.10.4.10:9200/'             # elasticsearch 접속 url
ES_INDEX_NAME = 'mpower_doc_128d_1'            # elasticsearch 인덱스명
ES_INDEX_FILE = './data/mpower10u_128d_1.json' # 인덱스 생성 파일
SEARCH_SIZE = 5                                # 검색 계수
MIN_SCORE = 1.4                                # 검색 1.4 스코어 이하면 제거

# 임베딩 모델 param
MODEL_PATH = '../../data11/model/kpf-sbert-128d-v1'
POLLING_MODE = 'mean'

# LLM 모델 param
lora_weights:str = '../../data11/model/LLM/beomi/KoAlpaca-Polyglot-5.8B/'      # lora weight 경로
llm_model_path:str ='../../data11/model/LLM/beomi/KoAlpaca-Polyglot-5.8B/'     # llm 모델경로(KoAlpaca-Polyglot-5.8B 모델이 가장 속도가 빠르고, 잘 응답하는것 같음.)
uselora_weight = False # Lora 사용하는 경우 True
load_8bit = True       # 8bit 로딩 

# prompt_template(모델에 따라 변경)
PROMPT_DICT = {
    #"prompt_context":("아래 내용을 가지고 질문에 대해 간략히 답변해 주세요\n\n### 내용: {context}\n\n### 질문: {query}\n\n### 답변:"),
    #"prompt_no_context":("질문에 대해 간략히 답변해 주세요\n\n### 질문: {query}\n\n### 답변:")
     
    "prompt_context":("### 질문: {query}\n\n아래 내용을 가지고 질문에 대해 간략히 답변해 주세요\n\n### 내용: {context}\n\n### 답변:"),
    "prompt_no_context":("### 질문: {query}\n\n질문에 대해 간략히 답변해 주세요\n\n### 답변:")
}
#-------------------------------------------

# prompt 테스트 
print(f'\n\nprompt 테스트----------------------\n')
query='제주도의 크기는 얼마?'
#context = ''
context = '제주도는 최남단에 있는 섬으로, 길이는 40km가 되는 대한민국에서 가장큰 섬이다.'
if context:
    prompt = PROMPT_DICT['prompt_context'].format(query=query, context=context)
else:
    prompt = PROMPT_DICT['prompt_no_context'].format(query=query)
    
print(prompt)

In [ ]:
#----------------------------------------------------------------------
# 임베딩 모델 로딩
#----------------------------------------------------------------------
from tqdm.notebook import tqdm
from myutils import embed_text, bi_encoder, mpower_index_batch


WORD_EMBDDING_MODEL1, BI_ENCODER1 = bi_encoder(model_path=MODEL_PATH, max_seq_len=512, do_lower_case=True, 
                                               pooling_mode=POLLING_MODE, out_dimension=OUT_DIMENSION, device=DEVICE)

print(BI_ENCODER1)

In [ ]:
#----------------------------------------------------------------------
# sLLM 모델 로딩
#----------------------------------------------------------------------
import torch
import transformers
from peft import PeftModel
import time

start_time = time.time()

# tokenizer 로딩
tokenizer = transformers.AutoTokenizer.from_pretrained(llm_model_path)

# 원본 모델 로딩
model = transformers.AutoModelForCausalLM.from_pretrained(llm_model_path, load_in_8bit=load_8bit, torch_dtype=torch.float16, device_map="auto")

if uselora_weight:
    model = PeftModel.from_pretrained(model, lora_weights, torch_dtype=torch.float16) # loRA 모델 로딩

if not load_8bit:
    model.half()
    
model.eval()

end_time = time.time() - start_time
print("time: {:.2f} ms\n".format(end_time * 1000)) 
print(model)

In [ ]:
# ES 관련
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import bulk

#--------------------------------------------
# 조건에 맞게 임베딩 처리하는 함수 
#--------------------------------------------
def embedding(paragraphs:list)->list:
    # 한 문단에 대한 40개 문장 배열들을 한꺼번에 임베딩 처리함
    embeddings = embed_text(model=BI_ENCODER1, paragraphs=paragraphs, return_tensor=False).astype(FLOAT_TYPE)    
    return embeddings


#------------------------------------------------------------------------
# ES임베딩 쿼리 벡터
# -쿼리 임베딩 벡터를 구하고, es에 접속해서 rfile_text 뽑아냄
#------------------------------------------------------------------------
def es_embed_query(query:str):
    assert query, f'query is empty'
    
    # 1.elasticsearch 접속
    es = Elasticsearch(ES_URL)  

    # 임베딩 구함.
    start_time = time.time()
    embed_query = embedding([query])[0]
    
    #print(len(embed_query))
    
    # 쿼리 구성
    script_query = {
        "script_score":{
            "query":{
                "match_all": {}},
            "script":{
                "source": "cosineSimilarity(params.query_vector, doc['vector1']) + 1.0",  # 뒤에 1.0 은 코사인유사도 측정된 값 + 1.0을 더해준 출력이 나옴
                "params": {"query_vector": embed_query}
            }
        }
    }
    
    #print(script_query)

    # 실제 ES로 검색 쿼리 날림
    start_search_time = time.time()
    response = es.search(
        index=ES_INDEX_NAME,
        body={
            "size": SEARCH_SIZE,
            "query": script_query,
            "_source":{"includes": ["rfile_name", "rfile_text"]}
        }
    )
    end_time = time.time() - start_time
    print("*ES 검색시간: {:.2f} ms".format(end_time * 1000)) 

    count = 0
    docs = []
    for hit in response["hits"]["hits"]: 
        doc = {}  #dict 선언
        doc['rfile_name'] = hit["_source"]["rfile_name"]      # contextid 담음
        doc['rfile_text'] = hit["_source"]["rfile_text"]      # text 담음.
        doc['score'] = hit["_score"]
        docs.append(doc) 
            
    return docs
#------------------------------------------------------------------------


In [ ]:
from transformers import GenerationConfig

#--------------------------------------------
# prompt 생성
#--------------------------------------------
def make_prompt(docs, query)->str:
     # prompt 구성
    context:str = ''

    for doc in docs:
        score = doc['score']
        if score > MIN_SCORE:
            rfile_text = doc['rfile_text']
            if rfile_text:
                context += rfile_text + '\n\n'
                
    if context:
        prompt = PROMPT_DICT['prompt_context'].format(query=query, context=context)
    else:
        prompt = PROMPT_DICT['prompt_no_context'].format(query=query)
                
    # KoAlpaca 프롬프트
    #prompt = f"### 질문: {query}\n질문에 대해 아래 내용을 바탕으로 간략히 답변해 주세요\n\n### 문맥: {context}\n\n### 답변:" if context else f"### 질문: {query}\n질문에 대해 간략히 답변해 주세요\n\n### 답변:"
    
    # llama 프롬프트
    #prompt = f"아래는 작업을 설명하는 명령어입니다. 요청을 적절히 완료하는 응답을 작성하세요. ### Instruction: {context}\n{query} ### Response:"

    #print(prompt)
    #print()
    return prompt
    
#-----------------------------------------
# text 생성
#-----------------------------------------
def generate_text(prompt):
    
    max_new_tokens = 256
    eos_str = tokenizer.decode(tokenizer.eos_token_id)
    start_time = time.time()

    #prompt = query
    #prompt = f"### 질문: {input_text}\n\n### 맥락: {context}\n\n### 답변:" if context else f"### 질문: {input_text}\n\n### 답변:"
    #prompt = f"### 질문 : 간략히 답변해줘.{query}\r\n###답변:"
    #prompt = f"### 질문: {query}\n\n### 답변:"

    #print(prompt)

    # config 설정
    generation_config = GenerationConfig(
        temperature=0.5,
        #top_p=0.75,
        #top_k=40,
        #num_beams=1,
        bos_token_id=tokenizer.bos_token_id,  # 시작토큰 
        eos_token_id=tokenizer.eos_token_id,  # end 토큰
        pad_token_id=tokenizer.pad_token_id   # padding 토큰
    )

    # 프롬프트 tokenizer 
    inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    input_ids = inputs["input_ids"]
    #print(input_ids)

    # Without streaming
    # generate 처리
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=False,
            max_new_tokens=max_new_tokens,
        )

    # 출력
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)

    end_time = time.time() - start_time
    print("*Text생성시간: {:.2f} ms\n".format(end_time * 1000)) 
    #print(output.replace(eos_str, ''))
    #print()
    return output.replace(eos_str, '')

In [ ]:
# 쿼리 입력후 테스트
def run_query_loop():
    while True:
        try:
            handle_query()
        except KeyboardInterrupt:
            return

def handle_query():
    # 쿼리는 일반 쿼리인 경우 '일반##제주도 면적은 얼마?'
    #query = '일반##제주도 면적은 얼마?' 

    query = input("질문:")
    
    query_split = query.split('##')
    prefix = query_split[0]
    #print(f'prefix: {prefix}')
    if prefix == '일반':
        query1 = query_split[1]
        prompt = make_prompt(docs='', query=query1)
        print(generate_text(prompt))
        print()
    else:
        query1 = query
        docs = es_embed_query(query1)
        print(docs)
        print()
        prompt = make_prompt(docs=docs, query=query1)
        
        print(generate_text(prompt))
        print()
        
run_query_loop()